## Дополнительная глава (4)

В данном блокноте собраны функции, позволяющие обработать наборы данных о посещениях сайтов пользователями. Сырые данные для этой задачи представляют собой cvs-файлы с данными о веб-сёрфинге отдельного пользователя в следующем виде: 
```
timestamp,site
2013-11-15 11:40:34,google.com
...
```
Требуется объединить коллекцию таких файлов в одну большую таблицу. При этом в полученной таблице отдельные строки - это сессии - последовательности из нескольких сайтов. Решение должно поддерживать создание таблицы с разной длиной сессии, а также с различной шириной окна.

**Пример**: для длины сессии 10 и ширины окна 7 файл из 30 записей породит не 3 сессии (1-10, 11-20, 21-30), а 5 (1-10, 8-17, 15-24, 22-30, 29-30). При этом в предпоследней сессии будет один ноль, а в последней – 8 нолей.

<span style="color:red">Важное замечание!</span> Задача, поставленная в этом разделе не связана напрямую с решением предложенной организаторами соревнования на Kaggel. Задача решалась исключительно в факультативных целях и для того, чтобы осознать необходимость оптимизации кода.

In [1]:
import warnings
warnings.filterwarnings('ignore')
from glob import glob
import os
import pickle
from tqdm import tqdm
import numpy as np
import pandas as pd

import itertools
import re
import scipy.sparse as sp
import time

PATH_TO_DATA = os.path.join('initial_data', 'users')
PATH_TO_SITE_FREQ = os.path.join('initial_data', 'site_freq')

Посмотрим на файл с данными на примере пользователя user0128 из коллекции с 10 пользователями.

In [2]:
user31_data = pd.read_csv(os.path.join(PATH_TO_DATA, '10users/user0128.csv'))
user31_data.head()

,timestamp,site
0,2013-11-15 13:46:03,fpdownload2.macromedia.com
1,2013-11-15 13:46:13,mail.google.com
2,2013-11-15 13:46:13,www.gmail.com
3,2013-11-15 13:46:25,accounts.google.com
4,2013-11-15 13:46:28,accounts.youtube.com


Основная функция в этом разделе - это функция
```
prepare_sparse_train_set_window.
```
Она принимает на вход путь к коллекции csv-файлов с данными по каждому пользователю, путь к заранее подготовленному словарю сайтов, а также параметры session_length и window_size, отвечающие за длину сессии и размер окна соответственно.

Для удобства, отдельно определена функция
```
make_sparse_data,
```
которая помогает на основе таблицы с сессиями создаёт матрицу частот в разреженном формате.

In [3]:
def make_sparse_data(data):
    ''' Принимает на вход DataFrame с сессиями и 
    возвращает разреженную матрицу частот'''
    
    indptr = [0]
    indices = []
    sparse_data = []
    for row in tqdm(data):
        val, cnt = np.unique(row[row != 0], return_counts=True)
        indptr.append(indptr[-1] + len(val))
        for v, c in zip(val, cnt):
            indices.append(v - 1)
            sparse_data.append(c)
            
    return np.uint64(sparse_data),  np.uint64(indices), np.uint64(indptr)

In [4]:
def prepare_data_set_window(path_to_csv_files, site_freq_path, 
                            session_length=10, window_size=10):
    ''' Подготавливает набор данных. Сырые данных хранятся в path_to_csv_files. 
    Возвращает матрицу частот в разреженном формате, вектор с user_id и DataFrame
    с соответствующим набором сессий'''
    
    with open(site_freq_path, 'rb') as file:
        site_freq_dict = pickle.load(file)
 
    data = []
    
    list_of_files = glob(os.path.join(path_to_csv_files, '*'))         
    for path_to_user in list_of_files:
        user_id = int(re.sub(r'\b0+', '', path_to_user[-8:-4], 1)) # user0001 --> 1
        sites_array = pd.read_csv(path_to_user)['site'].map(lambda x: site_freq_dict[x][0]).values.tolist()
        n = len(sites_array)
        ind = 0
        while True:
            if ind + session_length > n-1:
                data.append(sites_array[ind:n] + [0 for _ in range(ind + session_length - n)] + [user_id])
            else:
                data.append(sites_array[ind:ind+session_length] + [user_id])
            ind += window_size
            if ind >= n:
                break
    
    feature_names=[f'site{i}' for i in range(1, session_length + 1)] + ['target']
    data_df = pd.DataFrame(data, columns=feature_names)
    target = np.array(data_df['target'].values, dtype='int16')
    
    '''Создание и заполнение разреженной матрицы частот'''
    X, y = data_df.iloc[:, :-1].values, data_df.iloc[:, -1].values
    sparse_data = sp.csr_matrix(make_sparse_data(X), dtype='int8')
     
    return sparse_data, target, data_df 

In [5]:
%%time
X_sparse_10users_s10_w7, y_10users_s10_w7, df = prepare_data_set_window(os.path.join(PATH_TO_DATA,'150users'), 
                                                    os.path.join(PATH_TO_SITE_FREQ,'site_freq_150users.pkl'),
                                                    session_length=10, window_size=7)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 195712/195712 [00:06<00:00, 30078.11it/s]


Wall time: 9.7 s


Обработаем наборы данных для 10 и 150 пользователей и сохраним разреженные матрицы в формате 
```
X_sparse_{кол-во пользователей}users_s{длина сессии}_w{ширина окна}.pkl 
```
Также сохраним id пользователей в формате
```
y_{кол-во пользователей}users_s{длина сессии}_w{ширина окна}.pkl
```



In [6]:
%%time
for num_users in [10, 150]:
    for window_size, session_length in itertools.product([10, 7, 5], [15, 10, 7, 5]):
        if window_size <= session_length:
            X_sparse, y, _ = prepare_data_set_window(os.path.join(PATH_TO_DATA,'{}users'.format(num_users)),
                                                            os.path.join(PATH_TO_SITE_FREQ,'site_freq_{}users.pkl'.format(num_users)),
                                                            session_length=session_length, window_size=window_size)
            with open(os.path.join(PATH_TO_DATA, 
                                   'X_sparse_{}users_s{}_w{}.pkl'.format(num_users, 
                                                                         session_length, 
                                                                         window_size)), 'wb') as X_pkl:
                pickle.dump(X_sparse, X_pkl)
            with open(os.path.join(PATH_TO_DATA,
                                   'y_{}users_s{}_w{}.pkl'.format(num_users, 
                                                                  session_length, 
                                                                  window_size)), 'wb') as y_pkl:
                pickle.dump(y, y_pkl)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 273957/273957 [00:08<00:00, 32502.36it/s]


Wall time: 1min 39s


Теперь, когда подготовлены таблицы с сессиями в различных форматах (с различными session_length и window_size), можно попробовать натренировать модели (например, логистическую регрессию) на различных наборах данных. Тренировать модели на данных 150 пользователей долго, поэтому для примера, воспользуемся датасетами для 10 пользователей.

In [7]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import StratifiedKFold

def logit_cv(logit_c_values, path_to_X_pickle, path_to_y_pickle):
    ''' Тренирует LogisticRegressionCV с параметром из logit_c_values 
    на данных из path_to_X_pickle и path_to_y_pickle'''
    
    with open(path_to_X_pickle, 'rb') as X_sparse_users_pkl:
        X_sparse_users = pickle.load(X_sparse_users_pkl)
    with open(path_to_y_pickle, 'rb') as y_pkl:
        y = pickle.load(y_pkl)

    logit_c_values = np.linspace(0.5, 4, 5)

    logit_grid_searcher = LogisticRegressionCV(Cs=logit_c_values, 
                                               multi_class='multinomial', 
                                               random_state=17,
                                               cv=StratifiedKFold(n_splits=3, shuffle=True, random_state=17),
                                               n_jobs=-1)
    logit_grid_searcher.fit(X_sparse_users, y)

    return logit_grid_searcher

logit_c_values = np.linspace(0.5, 6, 5)

In [8]:
all_logit_searchers = []

for window_size, session_length in itertools.product([10, 7, 5], [15, 10, 7, 5]):
    if window_size <= session_length:
        path_to_X_pickle = os.path.join(PATH_TO_DATA,
                                     f'X_sparse_10users_s{session_length}_w{window_size}.pkl')
        path_to_y_pickle = os.path.join(PATH_TO_DATA, 
                                     f'y_10users_s{session_length}_w{window_size}.pkl')
        
        logit_grid_searcher = logit_cv(logit_c_values, path_to_X_pickle, path_to_y_pickle)
        
        all_logit_searchers.append(logit_grid_searcher)
        
        print(f'session_length = {session_length}, window_size = {window_size}')
        logit_mean_cv_scores = logit_grid_searcher.scores_[31].mean(axis=0)
        print('max cv score for user0031: ', logit_mean_cv_scores.max())
        print('best C: ', logit_grid_searcher.Cs_[logit_mean_cv_scores.argmax()])

session_length = 15, window_size = 10
max cv score for user0031:  0.8381338453879525
best C:  1.375
session_length = 10, window_size = 10
max cv score for user0031:  0.7763316976032999
best C:  1.375
session_length = 15, window_size = 7
max cv score for user0031:  0.8637924201474676
best C:  3.125
session_length = 10, window_size = 7
max cv score for user0031:  0.8123162268245263
best C:  1.375
session_length = 7, window_size = 7
max cv score for user0031:  0.7587494040411881
best C:  3.125
session_length = 15, window_size = 5
max cv score for user0031:  0.8841665915712135
best C:  3.125
session_length = 10, window_size = 5
max cv score for user0031:  0.8334162722580608
best C:  4.0
session_length = 7, window_size = 5
max cv score for user0031:  0.7878939587540709
best C:  3.125
session_length = 5, window_size = 5
max cv score for user0031:  0.7368590432312572
best C:  3.125


Наилучшее качество на кросс-валидации оказалось у датасета session_length = 15, window_size = 5. В дальнейшем можно проверить то же самое на более крупных датасетах, например на датасете 3000users. Но это выходит за рамки дополнительной главы, которая на этом заканчивается.